In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
class PlanarVAE(tf.keras.Model):
    """Planar Variational Autoencoder."""

    def __init__(self, latent_dim, input_shape_):
        super(PlanarVAE, self).__init__()
        self.latent_dim = latent_dim
        self.input_shape_ = input_shape_

        # Encoder
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=input_shape_),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(latent_dim + latent_dim)  # mean and log variance
        ])

        # Decoder
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(np.prod(input_shape_), activation=None),
            tf.keras.layers.Reshape(target_shape=input_shape_)
        ])

    def call(self, inputs):
        mean, logvar = self.encode(inputs)
        z = self.reparameterize(mean, logvar)
        logits = self.decode(z)
        return logits

    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        batch_size = tf.shape(mean)[0]
        eps = tf.random.normal(shape=(batch_size, self.latent_dim))
        return eps * tf.exp(logvar * .5) + mean

    def decode(self, z):
        logits = self.decoder(z)
        return logits

    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(100, self.latent_dim))
        return self.decode(eps)

In [4]:
# Load the time series from .npy files and subsample them
def load_data_from_folder(folder_path, subsample_factor=1):
    data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.npy'):
            file_path = os.path.join(folder_path, file_name)
            time_series = np.load(file_path)
            subsampled_time_series = time_series[::subsample_factor]
            data.append(subsampled_time_series)
    return np.array(data)

In [20]:
subsample_factor = 2
# Load data
folder_path = 'music_preprocessed_planarVAE'
data = load_data_from_folder(folder_path, subsample_factor=subsample_factor)

In [21]:
# Normalize data (assuming data is between 0 and 1; adjust if needed)
data = data.astype(np.float32) / np.max(data)

# Create TensorFlow dataset
batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices(data)
dataset = dataset.shuffle(buffer_size=len(data)).batch(batch_size)

In [22]:
latent_dim = 32  # Adjust as needed
input_shape_ = (1321967 // subsample_factor,)  # Adjusted input shape after subsampling

model = PlanarVAE(latent_dim, input_shape_)
optimizer = tf.keras.optimizers.Adam(1e-4)
model.build((None, *input_shape_))  # Build the model with input shape
model.summary()

Model: "planar_vae_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_8 (Sequential)   (None, 64)                338571584 
                                                                 
 sequential_9 (Sequential)   (None, 660983)            339224311 
                                                                 
Total params: 677795895 (-1583783716.00 Byte)
Trainable params: 677795895 (-1583783716.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# Ensure data shape matches the model's input shape
data = data[:, :input_shape_[0]]

# Create a TensorFlow dataset
batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices(data).batch(batch_size)

# Define the loss function and training step
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(
        -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
        axis=raxis)

def compute_loss(model, x):
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    x_logit = model.decode(z)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1])
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)

@tf.function
def train_step(model, x, optimizer):
    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Train the model
epochs = 5
for epoch in range(epochs):
    for batch in dataset:
        loss = train_step(model, batch, optimizer)
    print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}")